In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


# NYC Food Access — Neighborhood-Level (NTA2020) Analysis

This notebook focuses on a **single standardized neighborhood unit**: NYC Neighborhood
Tabulation Areas (NTA2020). We assume the **metrics CSV (with `nta2020` codes)** is the
source of truth for:
- Which neighborhoods exist
- Their IDs
- Metrics like supply gap, food insecurity, unemployment, etc.

In [42]:
CENSUS_TRACTS_GEOJSON = "raw/census_tracts.geojson"   # e.g., 2020 Census tracts GeoJSON/GeoPackage/SHAPE
PANTRIES_PATH      = "data/tabular/pantries.csv"    # can be a GeoJSON of points OR a CSV with lat/lon columns
SUPPLY_GAP_CSV     = "raw/Emergency_Food_Supply_Gap_20251110.csv"                    # e.g., "data/supply_gap_by_neighborhood.csv" or None

In [43]:

import pandas as pd
import geopandas as gpd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 60)
pd.set_option("display.width", 140)

# If PANTRIES_PATH is CSV, specify columns:
PANTRY_LAT_COL = "latitude"
PANTRY_LON_COL = "longitude"

CRS_LATLON    = "EPSG:4326"
CRS_PROJECTED = "EPSG:6539"  # NYC-appropriate projected CRS


In [44]:
tracts = gpd.read_file(CENSUS_TRACTS_GEOJSON)  # or your cleaned tract GeoJSON
# make sure it has at least ['geoid','nta2020','ntaname']
pantries = pd.read_csv(PANTRIES_PATH)
metrics = pd.read_csv(SUPPLY_GAP_CSV)

# 3️⃣ Turn pantries into a GeoDataFrame
gpantries = gpd.GeoDataFrame(
    pantries,
    geometry=[Point(xy) for xy in zip(pantries["lng"], pantries["lat"])],
    crs="EPSG:4326"   # WGS84 coordinates
)

# 4️⃣ Ensure coordinate systems match
tracts = tracts.to_crs(epsg=4326)

## 1. Aggregate NTA tracts

In [45]:
# Rename for consistency
metrics = metrics.rename(columns={
    "Neighborhood Tabulation Area NTA)": "nta2020",
    "Neighborhood Tabulation Area (NTA) Name": "ntaname"
})


In [46]:
metrics["nta2020"] = metrics["nta2020"].str.strip().str.upper()
metrics["Year"] = pd.to_datetime(metrics["Year"], format='%Y')

idx = metrics.groupby("nta2020")["Year"].idxmax()
metrics = metrics.loc[idx].reset_index(drop=True)

In [47]:
# 1️⃣ Keep only the NTA geometries from tracts (since each tract already has nta2020)
nta_geoms = (tracts[['nta2020', 'geometry']]
             .dissolve(by='nta2020')        # merge all tracts for same nta2020
             .reset_index())

print(f"Unique NTAs (from tracts): {nta_geoms.shape[0]}")

Unique NTAs (from tracts): 262


## Merge metrics and nta geometry gap

In [48]:
# 2️⃣ Join metrics onto those polygons
nta_enriched = nta_geoms.merge(metrics, on='nta2020', how='left')
print(f"NTA polygons after merge: {nta_enriched.shape}")

nta_enriched["Supply Gap (lbs.)"].isna().sum()

NTA polygons after merge: (262, 10)


65

In [49]:
nta_enriched.dropna(subset=['Supply Gap (lbs.)'], inplace=True)

## Pantries per tract and neighborhood

In [50]:
# 3️⃣ Spatially assign pantries to NTA polygons
pantries_in_nta = gpd.sjoin(gpantries, nta_enriched, how='left', predicate='within')
pantry_counts = (pantries_in_nta
                 .groupby('nta2020')
                 .size()
                 .rename('pantry_count')
                 .reset_index())

In [51]:
nta_enriched = nta_enriched.merge(pantry_counts, on='nta2020', how='left')
nta_enriched['pantry_count'] = nta_enriched['pantry_count'].fillna(0).astype(int)

In [52]:
# Export
nta_enriched.drop(columns=['geometry']).to_csv('data/tabular/nta_with_metrics.csv', index=False)
nta_enriched.to_file('data/geo/nta_with_metrics.geojson', driver='GeoJSON')

In [ ]:
# Centroids per neighborhood
nta_proj = nta_with_metrics.to_crs(CRS_PROJECTED)

# 2. Compute centroids in projected CRS
nta_proj["centroid_geom"] = nta_proj.geometry.centroid

# 3. Convert centroids back to WGS84 lat/lon for mapping/export
nta_centroids_ll = nta_proj.set_geometry("centroid_geom").to_crs(CRS_LATLON)

# 4. Extract simple coordinates
nta_centroids_ll["centroid_lon"] = nta_centroids_ll.geometry.x
nta_centroids_ll["centroid_lat"] = nta_centroids_ll.geometry.y

nta_centroids_ll.head()

In [37]:
nta_proj = nta_enriched.to_crs(CRS_PROJECTED)
centroids = nta_proj.copy()
centroids['geometry'] = nta_proj.centroid

centroids.head()

,nta2020,geometry,Year,Neighborhood Tabulation Area (NTA) Name,Supply Gap (lbs.),Food Insecure Percentage,Unemployment Rate,Vulnerable Population Score,Weighted Score,Rank,pantry_count
0,BK0101,POINT (998241.957 205056.938),2025-01-01,Greenpoint,"1,153,881.91747087",15.74%,741.51%,0.36,6.811714,44.0,1
1,BK0102,POINT (995721.016 199741.715),2025-01-01,Williamsburg,"1,011,421.07761282",16.48%,694.02%,0.38,6.326930,75.0,1
2,BK0103,POINT (996267.118 195489.486),2025-01-01,South Williamsburg,"2,090,950.72621075",27.58%,967.98%,0.60,8.033649,5.0,1
3,BK0104,POINT (1001857.171 199157.092),2025-01-01,East Williamsburg,"1,224,484.03196005",21.34%,739.65%,0.42,6.832984,43.0,1
4,BK0201,POINT (985683.594 192653.281),2025-01-01,Brooklyn Heights,"322,651.410235809",10.05%,407.41%,0.44,5.637125,139.0,0


In [ ]:
centroids.to_csv("data/tabular/nta_centroids.csv", index=False)

In [27]:
pantry_in_tract = gpd.sjoin(gpantries, tracts[["geoid","nta2020","ntaname","geometry"]],
                            how="left", predicate="within")

In [28]:
tract_pantry_counts = (
    pantry_in_tract.groupby("geoid")
    .size()
    .reset_index(name="pantry_count")
)


In [69]:
tracts = tracts.merge(metrics, on="nta2020", how="left")

In [40]:
tracts.nta2020.nunique()

262

In [72]:
tracts = tracts.merge(tract_pantry_counts, on="geoid", how="left")
tracts["pantry_count"] = tracts["pantry_count"].fillna(0)

In [73]:
nta_pantry_counts = (
    pantry_in_tract.groupby(["nta2020","ntaname"])
    .size()
    .reset_index(name="pantries_per_nta")
)